In [1]:
import numpy as np
from astropy.io import fits
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import itertools
from scipy.fft import ifftn, fftn
import imageio
from matplotlib.animation import FuncAnimation
from astropy.io import fits

In [29]:
def create_Te_cube(ref_cube, Te, R, resolution):

    '''
    Input:
    Ref cube: True 3D cube with emissivities.
    Te: True Te (T0) radial profile from the cloudy model grid
    R: True radius 
    resolution: 

    Output:
    

    '''
    ref_shape = ref_cube.shape       #cube_3D_cart.fits is the refernce cube
    cube = np.zeros(ref_shape)
    z = np.arange(ref_shape[0])
    y = np.arange(ref_shape[1])
    x = np.arange(ref_shape[2])
    center = np.divide(ref_shape, 2)
   
    r = R * resolution # resolution is in pix/parsec

    Te_interp = interp1d(r, Te, kind='cubic', fill_value=0, bounds_error=False) # shall we try fill_vlaue = 'extrapolate'? ne is zero in the halo, Te?

    factor = ref_shape[0]/ref_shape[2]
    
    for i in z:
        for j in y:
            for k in x:
                rad = np.sqrt((i-center[0])**2 + (j-center[1])**2 + factor**2*(k-center[2])**2)
                
                cube[i, j, k] = Te_interp(rad)

    return cube


In [30]:
#Radius’, 'Te', 'ne', 'H+', 'O0', 'O+', 'O++', 'N0', 'N+', 'N++', 'S0', 'S+', 'S++
simname = 'Bubble_v2_5e-14'

with fits.open('/home/amrita/LVM/lvmnebular/'+simname+'/testneb_tutorial3_ex1.fits') as hdu:
    vals=hdu['Comp_0_PhysParams'].data
R = vals[0][1:]
Te = vals[1][1:]

with fits.open('./Perturbation/cube_3D_cart.fits') as hdu:
    data=hdu[0].data

resolution = 1/0.777  # in pixels

cube=create_Te_cube(data, Te, R, resolution)

hdu1=fits.PrimaryHDU(cube)

hdul=fits.HDUList([hdu1])
hdul.writeto('./Perturbation/3d_Te_cube.fits', overwrite='True')


3037.8417835614014


In [14]:
# From the next block we are making the perturbation cube

def k_vector(npoints):
    k1 = np.arange(npoints/2+1)
    k2 = np.arange(-npoints/2+1, 0)
    
    kvector = 2*np.pi/ npoints* np.concatenate([k1, k2])
    return kvector

def pk_vector_delta(kvector, kvector2, ref_shape, k0, dk0):
    
    npoints = len(kvector)
    kk = np.zeros(ref_shape)
    
    factor = ref_shape[2]/ref_shape[0]  

    for i, j, k in itertools.product(range(ref_shape[0]), range(ref_shape[1]), range(ref_shape[2])):
        kk[i, j, k] = np.sqrt(kvector[i]**2 + kvector[j]**2 + factor**2*kvector2[k]**2)
               
    
    pk=np.zeros_like(kk)
    sel=(kk > k0-dk0/2)*(kk < k0+dk0/2)
    pk[sel]=1
    
    hdu1=fits.PrimaryHDU(pk)

    hdul=fits.HDUList([hdu1])
    hdul.writeto('./Perturbation/power_vector_delta.fits', overwrite='True')

    #xx, yy, zz = np.mgrid[0:npoints, 0:npoints, 0:npoints]
    #r = np.sqrt((xx-npoints/2)**2 + (yy-npoints/2)**2 + (zz-npoints/2)**2)
    #mask = r > npoints/2
    #mask2 = r < 0.8 * npoints/2
    #pk[mask*mask2]=0
    #pk[mask]=0

    pk[0,0,0] = 0
    
    return pk

def field_delta(k0, dk0, ref_cube):
    ref_shape = ref_cube.shape
    
    npoints = ref_shape[0]
    k = k_vector(npoints)
    k2 = k_vector(ref_shape[2])

    pk = pk_vector_delta(k, k2, ref_shape, k0, dk0)
    Pk1 = np.zeros_like(pk)
    #Pk1 /= Pk1.sum()
    Pk1 = pk

    field=np.random.randn(*ref_shape)
    fft_field=fftn(field)
    
    pspect_field = np.sqrt(Pk1) * fft_field
    new_field = np.real(ifftn(pspect_field))
    
    return new_field


In [15]:
def pc2k(x, L=36):
    return L/(2*np.pi*x) # returns k-number corresponding to scale x in pc

def fsr2k(x, L=2):
    return L/(2*np.pi*x) # returns k-number corresponding to a fraction of the Stromgren Radius

def pertsim(ref_cube, k0=fsr2k(0.5), dk0=0.05, Amp=0.1):
             
    new_field=field_delta(k0, dk0, ref_cube)
                             
    norm_field=new_field/np.std(new_field)*Amp

    return norm_field

In [33]:
def compute_pert_CEL_emissivity(data, T0, Tp):

    line = np.array([9532, 9069, 7319, 7320, 7330, 7331, 6731, 6716, 6584, 6548, 6312, 5755, 5007, 4959, 4363, 4069, 4076, 3970, 3729, 3726])
    em_line = np.array([138, 132, 101, 102,  103,  104,  96,   95,    93,   91,    89,  86,  72,  70,    45,   36,   37,   33,  17,  16])

    output = data.copy()

    for i,j in zip(line, em_line):
        chi = 6.626e-24*299792458 * 1e7/i      
        E0 = data[j, :, :, :]

        k = 1.38e-23
        A = -chi/(k*Tp)
        B = chi/(k*T0)
        C = np.sqrt(Tp)

        output[j, :, :, :] = np.exp(A+B)/C * np.sqrt(T0)* E0

    return output

In [40]:
def compute_pert_RL_emissivity(data, T0, Tp):

    line = np.array([4861, 6563])
    em_line = np.array([65, 92])

    output = data.copy()

    for i,j in zip(line, em_line):

        E0 = data[j, :, :, :]
        
        output[j, :, :, :]  = T0/Tp *E0 
    
    return output

In [46]:
n = 6
frac = np.linspace(6*10**(-2), 1, n)


with fits.open('./Perturbation/cube_4D_cart.fits') as hdul:
     data=hdul[0].data
     header = hdul[0].header

print(data.shape)
for i in frac:

    Amp =  np.linspace(0.05, 0.2, n)
    
    for j in Amp:

        perturbed_cube = pertsim(cube, k0=fsr2k(i), dk0 = i*0.1, Amp = j) # cube is the 3D_cart_cube and this function will give out perturbed 3D_cart_cube, ################# T0 #################
    
        #hdu1=fits.PrimaryHDU(perturbed_cube)
    
        #hdul=fits.HDUList([hdu1])
        #hdul.writeto('./Perturbation/'+str(i)+'_'+str(j)+'_perturbed_cube.fits', overwrite='True')
    
        pert_temp_cube = cube*(1+perturbed_cube)                                                             ################### 3D Tp ##################

        l = compute_pert_CEL_emissivity(data, cube, pert_temp_cube)                 
        l = compute_pert_RL_emissivity(l, cube, pert_temp_cube)     

        hdu=fits.PrimaryHDU(data = l, header = header)  

        hdu.writeto('./Perturbation/pert_em_cubes/'+str(i)+'_'+str(j)+'_pert_Emis_cube.fits', overwrite='True')
    
        #hdu1=fits.PrimaryHDU(pert_temp_cube)
    
        #hdul=fits.HDUList([hdu1])
        #hdul.writeto('./Perturbation/'+str(i)+'_'+str(j)+'_pert_temp_cube.fits', overwrite='True')

(141, 49, 49, 100)


/tmp/ipykernel_4704/4219846461.py:13: RuntimeWarning: divide by zero encountered in divide
  A = -chi/(k*Tp)
/tmp/ipykernel_4704/4219846461.py:14: RuntimeWarning: divide by zero encountered in divide
  B = chi/(k*T0)
/tmp/ipykernel_4704/4219846461.py:17: RuntimeWarning: invalid value encountered in add
  output[j, :, :, :] = np.exp(A+B)/C * np.sqrt(T0)* E0
/tmp/ipykernel_4704/3310245009.py:12: RuntimeWarning: invalid value encountered in divide
  output[j, :, :, :]  = T0/Tp *E0
/tmp/ipykernel_4704/4219846461.py:17: RuntimeWarning: invalid value encountered in multiply
  output[j, :, :, :] = np.exp(A+B)/C * np.sqrt(T0)* E0


In [ ]:
#################################################################### IGNORE: Below this is the base code ###################################################################

In [ ]:
l = compute_pert_CEL_emissivity(cube, pert_temp_cube)              

hdu1=fits.PrimaryHDU(l)

hdul=fits.HDUList([hdu1])
hdul.writeto('./Perturbation/pert_Emis_cube.fits', overwrite='True')

l = compute_pert_RL_emissivity(cube, pert_temp_cube)              

hdu1=fits.PrimaryHDU(l)

hdul=fits.HDUList([hdu1])
hdul.writeto('./Perturbation/pert_Emis_cube.fits', overwrite='True')

TypeError: compute_pert_CEL_emissivity() missing 1 required positional argument: 'Tp'

In [ ]:
#Below this is a function written to compute perturbations in emissivities

def compute_pert_emissivity(chi, T0, Tp, E0):

    k = 1.38e-23
    chi, E0 
    A = -chi/(k*Tp)
    B = chi/(k*T0)
    C = np.sqrt(Tp)

    de = np.exp(A+B)/C * np.sqrt(T0)* E0

    return de 

In [ ]:
with fits.open('./Perturbation/cube_4D_cart.fits') as hdul:
    data=hdul[0].data

    E_5007= data[72, :, :, :]
    

l5007 = compute_pert_emissivity(3.96e-19, cube, pert_temp_cube, E_5007)              #chi = 3.96e-19 J for [OIII] 5007 line

hdu1=fits.PrimaryHDU(l5007)

hdul=fits.HDUList([hdu1])
hdul.writeto('./Perturbation/pert_Emis_cube.fits', overwrite='True')

'''
with fits.open('cube_4D_cart.fits') as hdul:
    data=hdul[0].data

l5007 = compute_pert_emissivity(3.96e-19, cube, pert_temp_cube)  #chi = 3.96e-19 J/K for [OIII] 5007 line; E0 in the compute_pert_emissivity argument will also be removed; 

hdu1=fits.PrimaryHDU(l5007)
hdul=fits.HDUList([hdu1])
hdul.writeto('./Perturbation/d5007.fits', overwrite='True')
'''

/tmp/ipykernel_155832/687243472.py:6: RuntimeWarning: divide by zero encountered in divide
  A = -chi/(k*Tp)
/tmp/ipykernel_155832/687243472.py:7: RuntimeWarning: divide by zero encountered in divide
  B = chi/(k*T0)
/tmp/ipykernel_155832/687243472.py:10: RuntimeWarning: invalid value encountered in add
  de = np.exp(A+B)/C * np.sqrt(T0)* E0


"\nwith fits.open('cube_4D_cart.fits') as hdul:\n    data=hdul[0].data\n\nl5007 = compute_pert_emissivity(3.96e-19, cube, pert_temp_cube)  #chi = 3.96e-19 J/K for [OIII] 5007 line; E0 in the compute_pert_emissivity argument will also be removed; \n\nhdu1=fits.PrimaryHDU(l5007)\nhdul=fits.HDUList([hdu1])\nhdul.writeto('./Perturbation/d5007.fits', overwrite='True')\n"

In [ ]:
with fits.open('./Perturbation/cube_4D_cart.fits') as hdul:
    data=hdul[0].data

line = np.array([4363, 4959, 5007, 5755, 6548, 6584])
em_line = np.array([45, 70, 72, 86, 91, 93])

for i,j in zip(line, em_line):

    chi = 6.626e-24*299792458/i
    em_cube_max = np.max(data[j, :, :, :])


    print('chi_'+str(i), chi, )
    print('em_line_'+str(j), em_cube_max)
    print()

line = np.array([4861, 6563])
em_line = np.array([65, 92])

for i,j in zip(line, em_line):

    chi = 6.626e-24*299792458/i
    em_cube_max = np.max(data[j, :, :, :])


    print('chi_'+str(i), chi, )
    print('em_line_'+str(j), em_cube_max)
    print()

In [ ]:
#Script to save 3D cubes into gifs for presentation

In [ ]:
# Below is the code to save gifs of all cubes as animations (colours can be changed later)
# Load the FITS cube

with fits.open('./Perturbation/pert_Emis_cube.fits') as cube_hdulist:
    cube_data = cube_hdulist[0].data
    cube_hdulist.close()

# Set parameters
frame_interval = 0.3  # Time interval between frames in seconds
output_filename = './Perturbation/perturbed_Emissivity_cube.gif'

# Create the figure and axis for the animation
fig, ax = plt.subplots()

# Function to update each frame
def update_frame(frame_idx):
    ax.clear()
    im = ax.imshow(cube_data[frame_idx], cmap='Oranges_r')
    ax.set_title(f'Frame {frame_idx}')
    ax.axis('off')

# Add color bar
#cbar = plt.colorbar(im, ax=ax)
#cbar.set_label('Temperature')

# Create the animation
num_frames = cube_data.shape[0]
animation = FuncAnimation(fig, update_frame, frames=num_frames, interval=frame_interval * 1000)

# Save the animation as a GIF
animation.save(output_filename, writer='imagemagick', fps=1 / frame_interval)

plt.close(fig)
print(f'Animation saved as {output_filename}')


MovieWriter imagemagick unavailable; using Pillow instead.


Animation saved as ./Perturbation/perturbed_Emissivity_cube.gif


In [ ]:
def compute_pert_emissivity(T0, Tp, line):

    with fits.open('./Perturbation/cube_4D_cart.fits') as hdul:
        data=hdul[0].data

    lines = np.array([4363, 4959, 5007, 5755, 6548, 6584, 6312, 9069, 9532])
    
    em_line = np.array([45, 70, 72, 86, 91, 93, 89, 132, 138])

    for i,j in zip(line, em_line):

        if np.any(i == lines):

            chi = 6.626e-24*299792458/i
            E0 = data[j, :, :, :]

            k = 1.38e-23
            A = -chi/(k*Tp)
            B = chi/(k*T0)
            C = np.sqrt(Tp)

            de = np.exp(A+B)/C * np.sqrt(T0)* E0

            print(line, i, j)

        else:
            em_line = np.array([65, 92])
            for j in em_line:

                E0 = data[j, :, :, :]
                de = T0/Tp          
                print(j)                                                    #emmisivity dependence on T for RLs

In [ ]:
l = compute_pert_emissivity(cube, pert_temp_cube, line = np.array([4363, 4959, 5007, 5755, 6548, 6584, 6312, 9069, 9532, 4861, 6563]))


UFuncTypeError: ufunc 'logical_and' did not contain a loop with signature matching types (None, <class 'numpy.dtype[str_]'>) -> None

In [ ]:
############################ junks ##############################

simname =['Bubble_v2_1e-8','Bubble_v2_1e-8_z_0.2']

s = simname[1].split('_')
if len(s)==5:
    z=s[4]
else:
     z=1
#if len(s)

print(z, len(s))